# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [29]:
# imports

import os
import io
import sys
import anthropic
import gradio as gr
import subprocess
import google.generativeai
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display


In [30]:
# Load environment variables from .env file
load_dotenv(override=True)

# API keys from environment
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_model = os.getenv('GOOGLE_MODEL')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL', 'deepseek-v3.1:671b-cloud')

# Verify API keys
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[:7]}...")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[:2]}...")
else:
    print("Google API Key not set")

if ollama_base_url:
    print(f"Ollama configured at: {ollama_base_url}")

OpenAI API Key loaded: sk-proj-...
Anthropic API Key loaded: sk-ant-...
Google API Key loaded: AI...
Ollama configured at: http://192.168.80.200:11434


In [31]:
# Initialize API clients
openai_client = OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)
genai = google.generativeai.configure(api_key=google_api_key)

# Initialize Ollama client (uses OpenAI-compatible API)
ollama_client = OpenAI(
    base_url=f"{ollama_base_url}/v1",
    api_key=ollama_api_key
)

print("All clients initialized successfully")

All clients initialized successfully


In [43]:
#OPENAI_MODEL = "gpt-5"
#CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
#GROK_MODEL = "grok-4"
#GEMINI_MODEL = "gemini-2.5-pro"

# Want to keep costs ultra-low? Uncomment these lines:

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-haiku-20241022"
GROK_MODEL = "grok-4-fast-non-reasoning"
GEMINI_MODEL = "gemini-2.5-flash-lite"

In [33]:
system_message = "Eres un asistente que reimplementa codigo python en C++ de alto rendimiento"
system_message += "para una maquina con sistema operativo Windows 11 Pro, Procesador 12th Gen Intel(R) Core(TM) i7-1255U (1.70 GHz)"
system_message += "Responde solo con codigo C++; no expliques tu trabajo mas alla de algunos comentarios"
system_message += "Presta atencion a los tipos de numeros para asegurar que no haya desbordamientos de int."
system_message += "La respuesta C++ debe producir una salida identica en el menor tiempo posible."

In [34]:
def user_prompt_for(python):
    user_prompt = "Reescribe este codigo python en C++ con la implementacion mas rapida posible que produzca una salida identica en el menor tiempo posible"
    user_prompt += "Responde solo con codigo C++; no expliques tu trabajo mas alla de algunos comentarios"
    user_prompt += "presta atencion a los tipos de numeros para asegurar que no haya desbordamientos de int (overflow)."
    user_prompt += "Recuerda incluir todos los paquetes de C++ necesarios, como iomanip"
    user_prompt += python
    return user_prompt

In [35]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [56]:
def write_output(cpp):
    # Limpia los marcadores de código markdown
    code = cpp.replace('```cpp','').replace('```','').strip()
    with open("optimized.cpp", "w", encoding="utf-8") as f:
        f.write(code)

In [57]:
def optimize_gpt(python):
    stream = openai_client.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [58]:
def optimize_claude(python):
    result = claude_client.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role":"user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [59]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [47]:
exec(pi)

Result: 3.141592656089
Execution Time: 17.380954 seconds


In [60]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(200000000, 4.0, 1.0) * 4.0;

    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [61]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

Result: 3.141592656089
Execution Time: 0.341902170000 seconds


In [52]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, long long param1, long long param2) {
    double result = 1.0;
    
    for (long long i = 1; i <= iterations; ++i) {
        long long j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(200'000'000, 4, 1) * 4;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> diff = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << diff.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [62]:
# Compila C++ y corre el ejecutable (en el contenedor Linux de JupyterLab)
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!./optimized

Result: 3.141592656089
Execution Time: 0.339879154000 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file based on the instructions from GPT.

Again, it's not crucial to do this step if you don't wish to!

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [ ]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [ ]:
compile_and_run()

In [ ]:
19.178207/0.082168

## OK let's try the other contenders!

In [ ]:
port(anthropic, CLAUDE_MODEL, pi)
compile_and_run()

In [ ]:
port(grok, GROK_MODEL, pi)
compile_and_run()

In [ ]:
port(gemini, GEMINI_MODEL, pi)
compile_and_run()


In [ ]:
print(f"""
In Ed's experiments, the performance speedups were:

4th place: Claude Sonnet 4.5: {19.178207/0.104241:.0f}X speedup
3rd place: GPT-5: {19.178207/0.082168:.0f}X speedup
2nd place: Grok 4: {19.178207/0.018092:.0f}X speedup
1st place: Gemini 2.5 Pro: {19.178207/0.013314:.0f}X speedup
""")